# Base imports and functions

In [ ]:
!pip install --upgrade tensorflow_hub

In [ ]:
!pip install --upgrade tensorflow_addons

In [ ]:
%%capture
!pip install wandb

In [ ]:
import tensorflow_addons as tfa

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
from sklearn.utils import class_weight
import pandas as pd
import datetime
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import classification_report
from PIL import Image


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from keras.applications.inception_v3 import preprocess_input, decode_predictions

In [ ]:
from keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")

In [ ]:
def view_random_image(target_dir, target_class):
  target_folder = target_dir + target_class
  random_img = random.sample(os.listdir(target_folder), 1)
  img = mpimg.imread(target_folder + "/" + random_img[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off")

  print(f"Image shape: {img.shape}")
  return img


In [ ]:
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving to: {log_dir}")
  return tensorboard_callback

In [ ]:
def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed.
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)

    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)
    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1

    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))

In [ ]:
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend();

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
def create_model (model_url, num_classes, width, height):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable = False,
                                           dtype=tf.float32,
                                           name="feature_extraction_layer",
                                           input_shape = (width,height,3))
  model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dense(num_classes, activation="sigmoid", name="output_layer")
  ])
  return model

In [ ]:
def pred_and_plot(model, filename, true_label, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = filename

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  for i in range(len(class_names)):
    print(f"Prediction: {class_names[i]} {pred[0][i]*100:.2f}%, True: {true_label[i]}")

  # predicted_labels = [class_names[i] for i in range(len(class_names)) if pred[0][i] >= 0.5]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.axis(False);

In [ ]:
def plot_multi_label_confusion_matrix(true_labels, predicted_labels, class_labels):
    # Calculate the confusion matrix
    conf_matrix = multilabel_confusion_matrix(true_labels, predicted_labels)

    num_classes = len(class_labels)
    num_cols = 3  # Fixed number of columns
    num_rows = (num_classes + num_cols - 1) // num_cols  # Calculate the number of rows needed for plotting

    # Plot the confusion matrix
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(5 * num_cols, 5 * num_rows))

    for i in range(num_rows):
        for j in range(num_cols):
            ax = axes[i, j]
            class_index = i * num_cols + j
            if class_index < num_classes:
                ax.imshow(conf_matrix[class_index], cmap='Blues', interpolation='nearest')
                ax.set_title('Macierz błędów dla {}'.format(class_labels[class_index]))
                ax.set_xticks([0, 1])
                ax.set_yticks([0, 1])
                ax.set_xticklabels(['Predicted: 0', 'Predicted: 1'])
                ax.set_yticklabels(['True: 0', 'True: 1'])

                for k in range(2):
                    for l in range(2):
                        ax.text(l, k, str(conf_matrix[class_index][k][l]), ha='center', va='center', color='red')
            else:
                ax.axis('off')  # Turn off empty subplots

    plt.tight_layout()
    plt.show()

In [ ]:
def showF1scores(model_experiment, test_gen, class_names):
    pred = model_experiment.predict(test_gen, verbose=1)
    preds = np.where(pred < 0.5, 0, 1)

    classification_report_dict = classification_report(test_gen.labels, preds, output_dict=True)

    class_f1_scores = {}
    # Loop through classification report items
    for k, v in classification_report_dict.items():
        if k == "micro avg":  # stop once we get to accuracy key
            break
        else:
            # Append class names and f1-scores to new dictionary
            class_f1_scores[class_names[int(k)]] = v["f1-score"]

    f1_scores = pd.DataFrame({"class_name": list(class_f1_scores.keys()),
                              "f1-score": list(class_f1_scores.values())}).sort_values("f1-score", ascending=False)

    fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figsize as needed
    scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values)
    ax.set_yticks(range(len(f1_scores)))
    ax.set_yticklabels(list(f1_scores["class_name"]))
    ax.set_xlabel("f1-score")
    ax.set_title("Metryka F1")
    ax.invert_yaxis()  # reverse the order

    autolabel(scores, ax)  # Call autolabel function with scores and ax objects

    plt.show()

def autolabel(rects, ax):
    """
    Attach a text label above each bar displaying its height (it's value).
    """
    for rect in rects:
        width = rect.get_width()
        ax.text(1.03 * width, rect.get_y() + rect.get_height() / 2.,
                f"{width:.2f}",
                ha='center', va='center')

In [ ]:
def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two model history objects.
    """
    # Get original history measurements
    acc = original_history.history["f1_score"]
    loss = original_history.history["loss"]

    print(len(acc))

    val_acc = original_history.history["val_f1_score"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["f1_score"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_f1_score"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    print(len(total_acc))
    print(total_acc)

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training f1_score')
    plt.plot(total_val_acc, label='Validation f1_score')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation f1_score')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

# Data import and preprocessing

In [ ]:
import zipfile

my_files = zipfile.ZipFile("/content/drive/MyDrive/data_one_hot.zip")

my_files.extractall()
my_files.close()

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Zebrafishmetadata_combined.csv", sep=',')
class_names = columns=["Curved spine", "Dead", "Edema", "Normal", "Unhatched", "Yolk deformation"]

In [ ]:
print(df.head())

In [ ]:
# Count occurrences of each class
class_counts = df.iloc[:, 1:].sum()

# Plot the distribution - Bar Plot
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
bars = class_counts.plot(kind='bar', color='skyblue')
plt.title('Rozkład klas')
plt.xlabel('Klasy')
plt.ylabel('Liczebność')
plt.xticks(rotation=45)

# Add counts on top of the bars
for bar in bars.patches:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Plot the distribution - Pie Chart
plt.subplot(1, 2, 2)
class_counts.plot(kind='pie', autopct='%1.1f%%', colors=['lightcoral', 'lightgreen'])
plt.title('Rozkład klas')
plt.ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
img_height = img_width = 384
batch_size = 32

datagen=ImageDataGenerator(
    rescale=1.0/255.,
 )

datagen_augmented=ImageDataGenerator(
    rescale=1.0/255.,
    rotation_range=0.5,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_data = datagen.flow_from_dataframe(
  dataframe=df[:3387],
  directory="/content/data_one_hot/images",
  x_col="Filename",
  y_col=columns,
  batch_size=batch_size,
  seed=42,
  shuffle=True,
  class_mode="raw",
  target_size=(img_height,img_width))

train_data_augmented=datagen_augmented.flow_from_dataframe(
  dataframe=df[:3387],
  directory="/content/data_one_hot/images",
  x_col="Filename",
  y_col=columns,
  batch_size=batch_size,
  seed=42,
  shuffle=True,
  class_mode="raw",
  target_size=(img_height,img_width))

validation_gen=datagen.flow_from_dataframe(
  dataframe=df[3387:3763],
  directory="/content/data_one_hot/images",
  x_col="Filename",
  y_col=columns,
  batch_size=batch_size,
  seed=42,
  shuffle=True,
  class_mode="raw",
  target_size=(img_height,img_width))

test_gen=datagen.flow_from_dataframe(
  dataframe=df[3763:],
  directory="/content/data_one_hot/images",
  x_col="Filename",
  y_col=columns,
  seed=42,
  shuffle=False,
  class_mode="raw",
  target_size=(img_height,img_width))

In [ ]:
def plot_images_grid(images_arr, title, rows=1):
    cols = len(images_arr) // rows if len(images_arr) % rows == 0 else len(images_arr) // rows + 1
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    fig.suptitle(title, fontsize=20)

    # Ensure axes is always iterable
    if rows == 1 and cols == 1:
        axes = np.array([axes])
    elif rows == 1 or cols == 1:
        axes = axes.flatten()

    for i, ax in enumerate(axes.flat):
        if i < len(images_arr):
            ax.imshow(images_arr[i])
            ax.axis('off')
        else:
            fig.delaxes(ax)  # Remove empty subplots

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to make room for the title
    plt.show()

# Load a batch of original images
original_images, _ = next(train_data)

# Select one image from the batch
image = original_images[0]

# Generate augmented images
augmented_images = [datagen_augmented.random_transform(image) for _ in range(4)]

# Plot the original image
plot_images_grid([image], 'Oryginalne zdjęcie')

# Plot the augmented images in two rows
plot_images_grid(augmented_images, 'Augmentacja', rows=2)


In [ ]:
# Function to decode one-hot encoded labels
def decode_labels(one_hot_labels, columns):
    decoded_labels = []
    for one_hot_label in one_hot_labels:
        decoded_label = [column for column, value in zip(columns, one_hot_label) if value == 1]
        decoded_labels.append(decoded_label)
    return decoded_labels

# Get a batch of images and labels
images, one_hot_labels = train_data.next()

# Decode one-hot encoded labels
columns = ["Curved spine", "Dead", "Edema", "Normal", "Unhatched", "Yolk deformation"]
decoded_labels = decode_labels(one_hot_labels, columns)

# Plot the images
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])

    # Create a string representation of the decoded labels for the current image
    label_str = ', '.join(decoded_labels[i])

    plt.title(label_str)
    plt.axis("off")
plt.show()

In [ ]:
augmented_images, augmented_labels = train_data_augmented.next()

In [ ]:
train_images, train_labels = train_data.next()

In [ ]:
test_images, test_labels = test_gen.next()

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'),
      tfa.metrics.F1Score(num_classes=8, average='macro')
]

In [ ]:
METRICS2 = [
      tfa.metrics.F1Score(num_classes=6, average='macro'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR')
]

In [ ]:
class_weights_calc = generate_class_weights(train_labels, multi_class=False, one_hot_encoded=True)
print(class_weights_calc)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# SimpleModel


In [ ]:
checkpoint_filepath = 'drive/MyDrive/Colab Notebooks/TransferLearning/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    verbose=1)

In [ ]:
model_earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
simple = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(384, 384, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Dense(6, activation='sigmoid')
])


simple.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=METRICS2)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20))

In [ ]:
history=simple.fit(
            train_data,
            epochs=30,
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            callbacks=model_earlystop_callback)

In [ ]:
simple.save('drive/MyDrive/Colab Notebooks/Simple_model/')

In [ ]:
plot_metrics(history)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10, 7), xlabel="epochs")

Evaluation

In [ ]:
simple_model1 = tf.keras.saving.load_model('drive/MyDrive/Colab Notebooks/Simple_model/')

In [ ]:
simple_model1.evaluate(test_gen)

Prediction

In [ ]:
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size
test_gen.reset()
pred=simple_model1.predict(test_gen,
steps=len(test_gen),
verbose=1)

Confusion Matrix

In [ ]:
preds = np.where(pred < 0.5, 0, 1)
plot_multi_label_confusion_matrix(test_gen.labels, preds, class_names)

In [ ]:
showF1scores(simple_model1,test_gen, class_names)

In [ ]:
print(classification_report(test_gen.labels, preds, target_names = class_names))

# SimpleModel - class_weights


In [ ]:
model_earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
checkpoint_filepath = 'drive/MyDrive/Colab Notebooks/TransferLearning/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    verbose=1)

In [ ]:
simple_model = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(384, 384, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Dense(6, activation='sigmoid')
])


simple_model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=METRICS2)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20))

In [ ]:
history_1=simple_model.fit(
            train_data,
            epochs=15,
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks=[model_earlystop_callback,model_checkpoint_callback])

In [ ]:
run.finish()

In [ ]:
simple_model.load_weights(checkpoint_filepath)

In [ ]:
simple_model.save('drive/MyDrive/Colab Notebooks/Simple_model-weights/')

In [ ]:
plot_metrics(history_1)

In [ ]:
pd.DataFrame(history_1.history).plot(figsize=(10, 7), xlabel="epochs")

Evaluation

In [ ]:
simple_model2 = tf.keras.saving.load_model('drive/MyDrive/Colab Notebooks/Simple_model-weights/')

In [ ]:
simple_model2.evaluate(test_gen)

Prediction

In [ ]:
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size
test_gen.reset()
pred=simple_model.predict(test_gen,
steps=len(test_gen),
verbose=1)

Confusion Matrix

In [ ]:
preds = np.where(pred < 0.5, 0, 1)
plot_multi_label_confusion_matrix(test_gen.labels, preds, class_names)

In [ ]:
showF1scores(simple_model,test_gen, class_names)

In [ ]:
print(classification_report(test_gen.labels, preds, target_names = class_names))

# Model - data augmentation, class_weights

In [ ]:
model_earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
checkpoint_filepath = 'drive/MyDrive/Colab Notebooks/TransferLearning/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    verbose=1)

In [ ]:
model_3 = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Dense(6, activation='sigmoid')  # Assuming 6 classes with multi-label classification
])

model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=METRICS2)

In [ ]:
history_3=model_3.fit(
            train_data_augmented,
            epochs=30,
            steps_per_epoch=len(train_data_augmented),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                          tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)])

In [ ]:
model_3.save('drive/MyDrive/Colab Notebooks/Model3_augm/')

In [ ]:
model3 = tf.keras.saving.load_model('drive/MyDrive/Colab Notebooks/Model3_augm/')

In [ ]:
plot_metrics(history_3)

In [ ]:
model3_results = model3.evaluate(test_gen)

14/14 [==============================] - 16s 1s/step - loss: 0.2812 - f1_score: 0.5720 - precision: 0.6856 - recall: 0.6382 - auc: 0.9178 - prc: 0.7220


In [ ]:
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size
test_gen.reset()
pred=model3.predict(test_gen,
steps=len(test_gen),
verbose=1)

14/14 [==============================] - 71s 5s/step


In [ ]:
random_number = random.randint(0, 31)

pred_and_plot(model=model3, filename=test_images[random_number], true_label=test_gen.labels[random_number], class_names= class_names)

In [ ]:
preds = np.where(pred < 0.5, 0, 1)
plot_multi_label_confusion_matrix(test_gen.labels, preds, class_names)

In [ ]:
showF1scores(model3,test_gen, class_names)

In [ ]:
print(classification_report(test_gen.labels, preds, target_names = class_names))

# Transfer-Learning ResNet

In [ ]:
model_earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.2),
    layers.RandomFlip("horizontal_and_vertical"),
], name="data_augmentation")

def build_model(num_classes):
    inputs = layers.Input(shape=(224, 224, 3))
    x = data_augmentation(inputs)
    model = keras.applications.ResNet50V2(include_top=False, input_tensor=x, weights="imagenet", classes=num_classes)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation="relu")(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="sigmoid", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="ResNet50V2")
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=METRICS2
    )
    return model

model = build_model(num_classes=6)
resnet_history = model.fit(train_data,
            epochs=15,
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks=model_earlystop_callback)

In [ ]:
plot_metrics(resnet_history)

In [ ]:
showF1scores(model, test_gen, class_names)

In [ ]:
model.save('drive/MyDrive/Colab Notebooks/ResNet_BestSoFar/')

In [ ]:
test_gen.reset()

In [ ]:
pred=model.predict(test_gen,
steps=len(test_gen),
verbose=1)
preds = np.where(pred < 0.5, 0, 1)
plot_multi_label_confusion_matrix(test_gen.labels, preds, class_names)

In [ ]:
print(classification_report(test_gen.labels, preds, target_names = class_names))

# Fine-tuning ResNet

In [ ]:
model_earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.2),
    layers.RandomFlip("horizontal_and_vertical"),
], name="data_augmentation")


inputs = layers.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
base_model = keras.applications.ResNet50V2(include_top=False, input_tensor=x, weights="imagenet", classes=6)

    # Freeze the pretrained weights
for layer in base_model.layers[:]:
  layer.trainable = False

    # Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation="relu")(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(6, activation="sigmoid", name="pred")(x)

# Compile
model = keras.Model(inputs, outputs, name="ResNet50V2")
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=METRICS2
)

resnet_history = model.fit(train_data,
            epochs=10,
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks = model_earlystop_callback)


In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
for layer in base_model.layers[:178]:
   layer.trainable = False
for layer in base_model.layers[178:]:
   layer.trainable = True

print('Last block of the conv_base is now trainable')

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=METRICS2)

resnet_history_FT = model.fit(
            train_data,
            epochs=30,
            initial_epoch=resnet_history.epoch[-1],
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks = model_earlystop_callback)

In [ ]:
plot_metrics(resnet_history_FT)

In [ ]:
model.save('drive/MyDrive/Colab Notebooks/ResNet_FT_BestSoFar/')

In [ ]:
model.save('drive/MyDrive/Colab Notebooks/ResNet_FT_BestSoFar.h5')

In [ ]:
resnet_FT = tf.keras.saving.load_model('drive/MyDrive/Colab Notebooks/ResNet_FT_BestSoFar.h5', compile=False)

In [ ]:
resnet_FT.summary()

In [ ]:
showF1scores(resnet_FT, test_gen, class_names)

In [ ]:
resnet_FT.evaluate(test_gen)

In [ ]:
pred=resnet_FT.predict(test_gen,
steps=len(test_gen),
verbose=1)

In [ ]:
print(classification_report(test_gen.labels, preds, target_names = class_names))

In [ ]:
preds = np.where(pred < 0.5, 0, 1)
plot_multi_label_confusion_matrix(test_gen.labels, preds, class_names)

# Transfer-Learning InceptionV3

In [ ]:
showF1scores(model2, test_gen, class_names)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.2),
    layers.RandomFlip("horizontal_and_vertical"),
], name="data_augmentation")

def build_model(num_classes):
    inputs = layers.Input(shape=(299, 299, 3))
    x = data_augmentation(inputs)
    model = keras.applications.InceptionV3(include_top=False, input_tensor=x, weights="imagenet", classes=num_classes)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation="relu")(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="sigmoid", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="InceptionV3")
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=METRICS2
    )
    return model

model3 = build_model(num_classes=6)
resnet_history3 = model.fit(train_data,
            epochs=15,
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks=model_earlystop_callback)

In [ ]:
pred=model.predict(test_gen, verbose=1)
preds = np.where(pred < 0.5, 0, 1)
plot_multi_label_confusion_matrix(test_gen.labels, preds, class_names)

In [ ]:
model3.evaluate(test_gen)

In [ ]:
plot_metrics(resnet_history3)

In [ ]:
showF1scores(model3, test_gen, class_names)

In [ ]:
print(classification_report(test_gen.labels, preds, target_names = class_names))

In [ ]:
model3.save('drive/MyDrive/Colab Notebooks/Inception_BestSoFar/')

# Fine-tuning InceptionV3

In [ ]:
model_earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
checkpoint_filepath = 'drive/MyDrive/Colab Notebooks/TransferLearning/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    verbose=1)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.2),
    layers.RandomFlip("horizontal_and_vertical"),
], name="data_augmentation")


inputs = layers.Input(shape=(299, 299, 3))
x = data_augmentation(inputs)
base_model = keras.applications.InceptionV3(include_top=False, input_tensor=x, weights="imagenet", classes=6)

    # Freeze the pretrained weights
for layer in base_model.layers[:]:
  layer.trainable = False

    # Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024, activation="relu")(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(6, activation="sigmoid", name="pred")(x)

# Compile
inception = keras.Model(inputs, outputs, name="ResNet50V2")
optimizer = keras.optimizers.Adam(learning_rate=0.001)
inception.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=METRICS2
)

inception_history = inception.fit(train_data,
            epochs=10,
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks = model_earlystop_callback)


In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
for layer in base_model.layers[:249]:
   layer.trainable = False
for layer in base_model.layers[249:]:
   layer.trainable = True

print('Last block of the conv_base is now trainable')

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
inception.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=METRICS2)

inception_history_FT = inception.fit(
            train_data,
            epochs=30,
            initial_epoch=inception_history.epoch[-1],
            steps_per_epoch=len(train_data),
            validation_data=validation_gen,
            validation_steps=len(validation_gen),
            class_weight=class_weights_calc,
            callbacks = [model_earlystop_callback, model_checkpoint_callback])

In [ ]:
inception.load_weights(checkpoint_filepath)

In [ ]:
plot_metrics(inception_history_FT)

In [ ]:
test_gen.reset()

In [ ]:
inception.save('drive/MyDrive/Colab Notebooks/Inception_FT_BestSoFar2/')

In [ ]:
inception.save('drive/MyDrive/Colab Notebooks/Inception_FT_BestSoFar2.h5')

In [ ]:
resnet_FT = tf.keras.saving.load_model('drive/MyDrive/Colab Notebooks/Inception_FT_BestSoFar2/')

In [ ]:
showF1scores(resnet_FT, test_gen, class_names)

In [ ]:
resnet_FT.evaluate(test_gen)

In [ ]:
pred=resnet_FT.predict(test_gen,
steps=len(test_gen),
verbose=1)

14/14 [==============================] - 102s 7s/step


In [ ]:
preds = np.where(pred < 0.5, 0, 1)
plot_multi_label_confusion_matrix(test_gen.labels, preds, class_names)

In [ ]:
print(classification_report(test_gen.labels, preds, target_names = class_names))